In [1]:
from xbbg import blp
import pdblp
import workdays
import datetime
import pandas as pd
import matplotlib.pyplot as plt
pd.set_option('display.max_columns', 70)

In [2]:
T = ["SPXT Index","ES1 Index","SBUS13L Index","SBUS13YC Index","JPY BGN Curncy"]
d_from = "20070301"
d_to = workdays.workday(datetime.datetime.today(), days=-1).strftime("%Y%m%d")

# BDH = blp.bdh(T, "px_last", d_from, d_to, Days="W", Fill="P" ).reset_index()
BDH = blp.bdh(T, "px_last", d_from, d_to, Calendar="5D", Fill="P" ).reset_index()



BDH = BDH[["index"]+T]
df = pd.DataFrame(BDH["index"].values,columns = ["Date"])

def ME(df):
    df["ME"] = 0
    for i in range(1,len(df)):
        if df.loc[i,"Date"].month != df.loc[i-1,"Date"].month:
            if df.loc[i-1,"Date"].month ==3:
                df.loc[i-1,"ME"] = 3
            else:
                df.loc[i-1,"ME"] = 1
    df.loc[len(df)-1,"ME"] = 3    
    return df

df = ME(df)


df["SPXT"] = BDH["SPXT Index"].pct_change()
df["JPY"] = BDH["JPY BGN Curncy"].pct_change()
df["SPXT_JPY"] = (BDH["SPXT Index"] * BDH["JPY BGN Curncy"]).pct_change()

df["ES"] = BDH["ES1 Index"].pct_change()
df["HC"] = BDH["SBUS13YC Index"].pct_change() - BDH["SBUS13L Index"].pct_change()

df["ES+JPY"] = df["ES"] - df["HC"] + df["JPY"] 


df["SPXT_JPY_Index"] = 100
df.loc[1:,"SPXT_JPY_Index"] = ( 1 + df.loc[1:,"SPXT_JPY"] ).cumprod()*100

df["ES+JPY_Index"] = 100
df.loc[1:,"ES+JPY_Index"] = ( 1 + df.loc[1:,"ES+JPY"] ).cumprod()*100

df["HC_Index"] = 100
df.loc[1:,"HC_Index"] = ( 1 + df.loc[1:,"HC"] ).cumprod()*100

df

,Date,ME,SPXT,JPY,SPXT_JPY,ES,HC,ES+JPY,SPXT_JPY_Index,ES+JPY_Index,HC_Index
0,2007-03-01,0,NaN,NaN,NaN,NaN,NaN,NaN,100.000000,100.000000,100.000000
1,2007-03-02,0,-0.011400,-0.006972,-0.018292,-0.013708,-0.000140,-0.020539,98.170784,97.946074,99.985959
2,2007-03-05,0,-0.009364,-0.010873,-0.020136,-0.009739,-0.000393,-0.020219,96.194061,95.965720,99.946619
3,2007-03-06,0,0.015536,0.009262,0.024942,0.016942,-0.000114,0.026318,98.593323,98.491309,99.935183
4,2007-03-07,0,-0.002182,-0.004631,-0.006803,-0.001608,-0.000135,-0.006105,97.922561,97.890059,99.921729
...,...,...,...,...,...,...,...,...,...,...,...
4317,2023-09-18,0,0.000737,-0.001623,-0.000887,0.000778,-0.000488,-0.000357,555.148174,490.692993,77.542784
4318,2023-09-19,0,-0.002152,0.001694,-0.000462,-0.002555,-0.000173,-0.000688,554.891645,490.355296,77.529381
4319,2023-09-20,0,-0.009327,0.003246,-0.006111,-0.009577,-0.000175,-0.006155,551.500481,487.337132,77.515777
4320,2023-09-21,0,-0.016368,-0.005056,-0.021341,-0.016865,-0.000151,-0.021770,539.730859,476.727664,77.504074
